In [ ]:
# Articles
# - https://jyx.jyu.fi/handle/123456789/78083

In [ ]:
# Notes

# Important paper  => Kania2021_Chapter_InteractiveMultiobjectiveOptim.pdf

In [78]:
import pandas as pd
import numpy as np

df = pd.read_excel('Demand_data_case.xlsx')

months = list(range(1, 109))
print(len(months))
df['months'] = months
df

print(df)

x1 = np.array(df['Demand (pcs)'])
x2 = np.array(df['months'])

x = pd.array([x1, x2])

108
     Month  Demand (pcs)  months
0    m_001            11       1
1    m_002            12       2
2    m_003            10       3
3    m_004             8       4
4    m_005            12       5
..     ...           ...     ...
103  m_104             3     104
104  m_105             0     105
105  m_106             0     106
106  m_107             3     107
107  m_108             5     108

[108 rows x 3 columns]


In [66]:
# Objectives

def f1(x):
    return x[0]+x[1]#.reshape(-1,1)

def f2(x):
    return x[0]*x[1]#.reshape(-1,1)

f_1 = f1(x)
f_2 = f2(x)

print(f_1)


[ 12  14  13  12  17  15  19  11  14  16  15  21  23  14  15  16  17  22
  25  32  30  34  23  30  41  50  35  45  32  32  31  32  35  38  45  43
  47  40  44  45  53  42  52  46  45  54  49  51  65  50  63  56  57  56
  61  56  57  62  63  64  73  74  81  66  74  70  75  68  72  72  71  76
  81  80  75  87  84  82  93  80  81  82  85  87 101  88  95  98  89  96
  91  92  93  94  97  96  97  98  99 105 112 107 107 107 105 106 110 113]


In [3]:

# Safety stock

R = np.mean(x[0])
std_R = np.std(x[0])
L = 12 # lead time - 12 months
#sL = 0 # lead time standard deviation
CSL = 0.9 # cycle service level

SS = (CSL * std_R * L)**0.5
print('safety stock', SS)

print('Demand mean / average', R)
print('Demand standard deviation', std_R)

safety stock 7.410282970677182
Demand mean / average 5.333333333333333
Demand standard deviation 5.084471639398911


In [81]:
# Surrogate
from desdeo_problem import VectorDataObjective as VDO
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

#data = np.hstack((x, f_1, f_2))
#print(data)

data = {
    'x1': x1,
    'x2': x2,
    'f1': f_1,
    'f2': f_2
}

                             # demand , months
datapd = pd.DataFrame(data=data) # columns=['x1', 'x2', 'f1', 'f2']

print(datapd)
obj = VDO(data=datapd, name=['f1', 'f2'])

obj.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})
 
#print(obj.evaluate(np.asarray([[0.5,0.3]]), use_surrogate=True))

     x1   x2   f1   f2
0    11    1   12   11
1    12    2   14   24
2    10    3   13   30
3     8    4   12   32
4    12    5   17   60
..   ..  ...  ...  ...
103   3  104  107  312
104   0  105  105    0
105   0  106  106    0
106   3  107  110  321
107   5  108  113  540

[108 rows x 4 columns]


In [96]:
# Problem

from desdeo_problem import DataProblem

maximize = pd.DataFrame([[True, True]], columns=['f1','f2'])
prob = DataProblem(data=datapd, objective_names=['f1','f2'], variable_names=['x1', 'x2'], maximize=maximize)

prob.train(GaussianProcessRegressor)

data_to_evaluate = []
for i in range(len(x1)):
    data_to_evaluate.append([x1[i], x2[i]])


print(prob.evaluate(np.asarray(data_to_evaluate), use_surrogate=True))

Evaluation Results Object 
Objective values are: 
[[ 1.20000000e+01  1.10000000e+01]
 [ 1.40000000e+01  2.40000000e+01]
 [ 1.30000000e+01  3.00000000e+01]
 [ 1.20000000e+01  3.20000000e+01]
 [ 1.70000000e+01  6.00000000e+01]
 [ 1.50000000e+01  5.40000000e+01]
 [ 1.90000000e+01  8.40000000e+01]
 [ 1.10000000e+01  2.40000000e+01]
 [ 1.40000000e+01  4.50000000e+01]
 [ 1.60000000e+01  6.00000000e+01]
 [ 1.50000000e+01  4.40000000e+01]
 [ 2.10000000e+01  1.08000000e+02]
 [ 2.30000000e+01  1.30000000e+02]
 [ 1.40000000e+01 -2.59885863e-13]
 [ 1.50000000e+01  7.09980685e-13]
 [ 1.60000000e+01 -1.31478162e-12]
 [ 1.70000000e+01  1.98796882e-12]
 [ 2.20000000e+01  7.20000000e+01]
 [ 2.50000000e+01  1.14000000e+02]
 [ 3.20000000e+01  2.40000000e+02]
 [ 3.00000000e+01  1.89000000e+02]
 [ 3.40000000e+01  2.64000000e+02]
 [ 2.30000000e+01  1.41894112e-16]
 [ 3.00000000e+01  1.44000000e+02]
 [ 4.10000000e+01  4.00000000e+02]
 [ 5.00000000e+01  6.24000000e+02]
 [ 3.50000000e+01  2.16000000e+02]
 [ 4.

/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names
  warnings.warn(
